## Data Pipeline steps
1. Data Extraction
2. Quality Assessment
3. Data Cleaning
4. Data Validation
5. Featue Engineering
6. Loading
7. Analysis and insights

## Import libraries and select the kernel for virtual enviroment

In [65]:
#importing libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine    
from dotenv import load_dotenv
import psycopg2


In [66]:

# Define proper data types for key columns like 'Customer_ID', 'Order_ID', 'Order_Date', 'Product_Category', 'Product_Sub_Category' , etc.
dtypes_dict = {
    'customer_id': 'str',
    'order_id': 'str',
    'order_date': 'str',
    'product_category': 'str',
    'product_sub_category': 'str'
}

In [67]:
df = pd.read_csv(r'C:\Users\David Ibanga\Data Engineering practicals\Globex_Retail\data\globex_retail_data.csv', dtype=dtypes_dict)

In [68]:
df.head()

,Customer_ID,Order_ID,Order_Date,Product_Category,Product_Sub_Category,Quantity,Price,Discount,Customer_Location,Revenue
0,CUST_013738,ORD_00102406,2023-01-01,Home & Garden,Gardening Tools,1,419.19,0.00,TN,419.1900
1,CUST_011726,ORD_00102902,2023-01-01,Electronics,Laptops,1,222.37,0.09,TN,202.3567
2,CUST_010891,ORD_00103864,2023-01-01,Electronics,Laptops,6,1107.65,0.00,IN,6645.9000
3,CUST_011452,ORD_00103560,2023-01-01,Electronics,Gaming Consoles,5,288.84,0.00,MA,1444.2000
4,CUST_010886,ORD_00100632,2023-01-02,Electronics,Headphones,1,191.27,0.00,AZ,191.2700


### Data Structure

In [69]:
df.columns

Index(['Customer_ID', 'Order_ID', 'Order_Date', 'Product_Category',
       'Product_Sub_Category', 'Quantity', 'Price', 'Discount',
       'Customer_Location', 'Revenue'],
      dtype='object')

### Step2: Data Quality Assessment checks
## Check for uniqueness, completeness and data consistency
### Checking for missing values

### Data Cleaning And Transformation

In [70]:
# Check for missing values in the dataset
missing_values = df.isnull().sum()
print("\n Missing values in dataset before cleaning:")
print(missing_values)



 Missing values in dataset before cleaning:
Customer_ID             0
Order_ID                0
Order_Date              0
Product_Category        0
Product_Sub_Category    0
Quantity                0
Price                   0
Discount                0
Customer_Location       0
Revenue                 0
dtype: int64


In [71]:
# Check for missing values in key columns
key_cols = ['Customer_ID', 'Order_ID', 'Order_Date', 'Product_Category', 'Quantity', 'Price', 'Revenue']
missing_summary = df[key_cols].isnull().sum()
print("\n Missing values in key columns before cleaning:")
print(missing_summary)


 Missing values in key columns before cleaning:
Customer_ID         0
Order_ID            0
Order_Date          0
Product_Category    0
Quantity            0
Price               0
Revenue             0
dtype: int64


### Step3: Data Cleaning

In [72]:
# create a copy of original data for cleaning
df_clean = df.copy()

In [73]:
df_clean.head()

,Customer_ID,Order_ID,Order_Date,Product_Category,Product_Sub_Category,Quantity,Price,Discount,Customer_Location,Revenue
0,CUST_013738,ORD_00102406,2023-01-01,Home & Garden,Gardening Tools,1,419.19,0.00,TN,419.1900
1,CUST_011726,ORD_00102902,2023-01-01,Electronics,Laptops,1,222.37,0.09,TN,202.3567
2,CUST_010891,ORD_00103864,2023-01-01,Electronics,Laptops,6,1107.65,0.00,IN,6645.9000
3,CUST_011452,ORD_00103560,2023-01-01,Electronics,Gaming Consoles,5,288.84,0.00,MA,1444.2000
4,CUST_010886,ORD_00100632,2023-01-02,Electronics,Headphones,1,191.27,0.00,AZ,191.2700


### Checking for duplicates of data


In [74]:
#Check and remove duplicate rows
num_duplicates = df_clean.duplicated().sum()
if num_duplicates > 0:
    df_clean = df_clean.drop_duplicates()
    print(f"\n Removed {num_duplicates} duplicate rows.")
else:
    print("\n No duplicate rows found.")


 No duplicate rows found.


In [75]:
df_clean = df_clean.dropna(subset=key_cols)
print(f"\n Dropped rows with missing critical values. New shape: {df_clean.shape}")


 Dropped rows with missing critical values. New shape: (5000, 10)


In [76]:
# Convert Order_Date to datetime
df_clean['Order_Date'] = pd.to_datetime(df_clean['Order_Date'], errors='coerce')
print("\nAfter converting Order_Date to datetime:")
print(df_clean[['Order_Date']].head())



After converting Order_Date to datetime:
  Order_Date
0 2023-01-01
1 2023-01-01
2 2023-01-01
3 2023-01-01
4 2023-01-02


In [77]:
#Validate revenue calculation matches Quantity * Price * (1 - Discount)
df_clean['Calculated_Revenue'] = df_clean['Quantity'] * df_clean['Price'] * (1 - df_clean['Discount'])
revenue_diff = (df_clean['Revenue'] - df_clean['Calculated_Revenue']).abs().mean()
print(f"\n Average Revenue Calculation Difference: {revenue_diff:.4f}")


 Average Revenue Calculation Difference: 0.0000


## Step 4: Data Transformation
### Feature Engineering

In [78]:
# Created Order_Month from Order_Date
df_clean['Order_Month'] = df_clean['Order_Date'].dt.to_period('M').astype(str)
print("\nAfter engineering Order_Month:")
print(df_clean[['Order_Date', 'Order_Month']].head())


After engineering Order_Month:
  Order_Date Order_Month
0 2023-01-01     2023-01
1 2023-01-01     2023-01
2 2023-01-01     2023-01
3 2023-01-01     2023-01
4 2023-01-02     2023-01


In [79]:
print("\nCurrent columns before merge:")
print(df_clean.columns.tolist())



Current columns before merge:
['Customer_ID', 'Order_ID', 'Order_Date', 'Product_Category', 'Product_Sub_Category', 'Quantity', 'Price', 'Discount', 'Customer_Location', 'Revenue', 'Calculated_Revenue', 'Order_Month']


In [99]:
#  Create Order_Value column
df_clean['Order_Value'] = df_clean['Revenue']
df_clean.head()

,Customer_ID,Order_ID,Order_Date,Product_Category,Product_Sub_Category,Quantity,Price,Discount,Customer_Location,Revenue,Calculated_Revenue,Order_Month,Order_Value,Avg_Order_Value_Calc,Customer_Segment
0,CUST_013738,ORD_00102406,2023-01-01,Home & Garden,Gardening Tools,1,419.19,0.00,TN,419.1900,419.1900,2023-01,419.1900,571.198267,Regular
1,CUST_011726,ORD_00102902,2023-01-01,Electronics,Laptops,1,222.37,0.09,TN,202.3567,202.3567,2023-01,202.3567,202.356700,Regular
2,CUST_010891,ORD_00103864,2023-01-01,Electronics,Laptops,6,1107.65,0.00,IN,6645.9000,6645.9000,2023-01,6645.9000,6645.900000,High-Value
3,CUST_011452,ORD_00103560,2023-01-01,Electronics,Gaming Consoles,5,288.84,0.00,MA,1444.2000,1444.2000,2023-01,1444.2000,760.540000,Regular
4,CUST_010886,ORD_00100632,2023-01-02,Electronics,Headphones,1,191.27,0.00,AZ,191.2700,191.2700,2023-01,191.2700,1125.855000,High-Value


In [100]:
#  Compute average order value per customer
customer_avg_order = df_clean.groupby('Customer_ID')['Order_Value'].mean().reset_index()
customer_avg_order.columns = ['Customer_ID', 'Avg_Order_Value']
customer_avg_order

,Customer_ID,Avg_Order_Value
0,CUST_010001,1124.238500
1,CUST_010002,981.440000
2,CUST_010003,466.543500
3,CUST_010006,452.943867
4,CUST_010007,206.688000
...,...,...
3198,CUST_014993,2475.183400
3199,CUST_014994,694.935000
3200,CUST_014995,2129.200000
3201,CUST_014996,121.965000


In [82]:
#Renaming customer_avg_order column for merging
customer_avg_order.columns = ['Customer_ID', 'Avg_Order_Value_Calc']


In [83]:
# Merge back Avg_Order_Value to main df

if 'Avg_Order_Value' in df_clean.columns:
    df_clean = df_clean.drop(columns=['Avg_Order_Value'])

df_clean = df_clean.merge(customer_avg_order, on='Customer_ID', how='left')




In [84]:
print("\nColumns after merge:")
print(df_clean.columns.tolist())



Columns after merge:
['Customer_ID', 'Order_ID', 'Order_Date', 'Product_Category', 'Product_Sub_Category', 'Quantity', 'Price', 'Discount', 'Customer_Location', 'Revenue', 'Calculated_Revenue', 'Order_Month', 'Order_Value', 'Avg_Order_Value_Calc']


### Customer Segmentation

In [87]:
print("\nColumns in customer_avg_order:", customer_avg_order.columns.tolist())



Columns in customer_avg_order: ['Customer_ID', 'Avg_Order_Value']


In [88]:
#Segment customers into High-Value (top 20% average order value) vs Regular
high_value_threshold = customer_avg_order['Avg_Order_Value'].quantile(0.8)

df_clean['Customer_Segment'] = df_clean['Avg_Order_Value_Calc'].apply(
    lambda x: 'High-Value' if x >= high_value_threshold else 'Regular'
)
print(f"\n High-Value customer threshold: {high_value_threshold:.2f}")
print(df_clean[['Customer_ID', 'Avg_Order_Value_Calc', 'Customer_Segment']].head())



 High-Value customer threshold: 1049.82
   Customer_ID  Avg_Order_Value_Calc Customer_Segment
0  CUST_013738            571.198267          Regular
1  CUST_011726            202.356700          Regular
2  CUST_010891           6645.900000       High-Value
3  CUST_011452            760.540000          Regular
4  CUST_010886           1125.855000       High-Value


### Generating Insights with Pandas

In [ ]:
#Question 1: Product category with the most generated revenues
category_revenue = df_clean.groupby('Product_Category')['Revenue'].sum().sort_values(ascending=False)
print("\n 1) Revenue by Product Category:")
print(category_revenue)


 1) Revenue by Product Category:
Product_Category
Electronics      2.079574e+06
Home & Garden    6.814707e+05
Sports           4.744529e+05
Clothing         2.819485e+05
Beauty           1.371077e+05
Books            6.522193e+04
Name: Revenue, dtype: float64


In [ ]:
#Question 2: Product subcategory with the most generated revenues
subcategory_revenue = df_clean.groupby('Product_Sub_Category')['Revenue'].sum().sort_values(ascending=False)
print("\n 2) Revenue by Product Subcategory:")
print(subcategory_revenue.head(10))


 2) Revenue by Product Subcategory:
Product_Sub_Category
Smart Watches         403335.4208
Smartphones           365802.8764
Tablets               348740.8234
Gaming Consoles       332296.2353
Headphones            318398.7129
Laptops               310999.7473
Gardening Tools       129829.3451
Kitchen Appliances    125048.8293
Storage               110330.1389
Home Decor            109826.6465
Name: Revenue, dtype: float64


In [ ]:
## Question 3: Customer segment with the highest average order value
segment_avg_order = df_clean.groupby('Customer_Segment')['Order_Value'].mean().sort_values(ascending=False)
print("\n 3) Average Order Value by Customer Segment:")
print(segment_avg_order)


 3) Average Order Value by Customer Segment:
Customer_Segment
High-Value    2287.693006
Regular        352.214206
Name: Order_Value, dtype: float64


In [92]:
## Question 4: Average discount applied to high-value customers
high_value_discount = df_clean[df_clean['Customer_Segment'] == 'High-Value']['Discount'].mean()
print(f"\n 4) Average Discount for High-Value Customers: {high_value_discount:.2%}")



 4) Average Discount for High-Value Customers: 4.91%


In [93]:
## Question 5: Geographic locations with most high-value customers
high_value_customers = df_clean[df_clean['Customer_Segment'] == 'High-Value']
location_counts = high_value_customers['Customer_Location'].value_counts()
print("\n 5) High-Value Customer Locations (Top 5):")
print(location_counts.head(5))


 5) High-Value Customer Locations (Top 5):
Customer_Location
CO    48
LA    47
MD    42
IL    41
NC    41
Name: count, dtype: int64


In [103]:
## Question 6:How Discount Rate Impact Total Sales across different product categories
discount_impact = df_clean.groupby('Product_Category').apply(
    lambda x: pd.Series({
        'Avg_Discount': x['Discount'].mean(),
        'Total_Revenue': x['Revenue'].sum()
    })
).sort_values('Total_Revenue', ascending=False)
print("\n 6) Discount Rate Impact by Product Categories:")
print(discount_impact)


 6) Discount Rate Impact by Product Categories:
                  Avg_Discount  Total_Revenue
Product_Category                             
Electronics           0.058537   2.079574e+06
Home & Garden         0.063398   6.814707e+05
Sports                0.069806   4.744529e+05
Clothing              0.063377   2.819485e+05
Beauty                0.061279   1.371077e+05
Books                 0.065040   6.522193e+04


C:\Users\David Ibanga\AppData\Local\Temp\ipykernel_34416\2575364459.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  discount_impact = df_clean.groupby('Product_Category').apply(


In [95]:
## Question 7: Monthly purchase trends
monthly_trends = df_clean.groupby('Order_Month')['Revenue'].sum().sort_index()
print("\n 7) Monthly Revenue Trends:")
print(monthly_trends)



 7) Monthly Revenue Trends:
Order_Month
2023-01    177437.6013
2023-02    173608.8882
2023-03    151549.1629
2023-04    160107.8262
2023-05    167926.8782
2023-06    134903.8958
2023-07    202245.1529
2023-08    102365.7004
2023-09    153440.8570
2023-10    160409.8015
2023-11    137063.2309
2023-12    154119.1079
2024-01    187767.1758
2024-02    185474.4577
2024-03    156389.5161
2024-04    133587.1201
2024-05    164528.5586
2024-06    140662.3017
2024-07    165830.4983
2024-08    130879.2232
2024-09    147267.8345
2024-10    133072.4755
2024-11    153979.2064
2024-12    145159.1055
Name: Revenue, dtype: float64


### DATA LOADING

In [98]:
import os
from urllib.parse import quote_plus

load_dotenv()
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')

# URL-encode the password to handle special characters like '@'
DB_PASSWORD_ENC = quote_plus(DB_PASSWORD)

# Create a connection string with encoded password
db_url = f'postgresql://{DB_USER}:{DB_PASSWORD_ENC}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
engine = create_engine(db_url)

# Load the DataFrame into PostgreSQL
df_clean.to_sql('globex_retail', engine, if_exists='replace', index=False)
print("Data loaded successfully into PostgreSQL database.")

Data loaded successfully into PostgreSQL database.


#### Recommendations
#### Focus on Electronics: Since Electronics generates the most revenue, ensure optimal inventory levels for tablets, smartphones, and laptops.

#### Target High-Value Customers: Develop loyalty programs for high-value customers, especially in CA, NY, and TX.

#### Strategic Discounting: Use discounts more strategically in categories where they drive significant volume increases (like Clothing), rather than applying them uniformly.

### Seasonal Planning: Prepare inventory and marketing campaigns for peak seasons identified in the monthly trends.

### Subcategory Optimization: Within top categories, focus on the highest-performing subcategories while evaluating underperforming ones.